# Scratch Work for Time Series Modeling

In [ ]:
import os
import warnings

import numpy as np
import pandas as pd
from pmdarima.arima import AutoARIMA
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
import statsmodels.api as sm

from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
import plotnine as p9
import seaborn as sns

warnings.filterwarnings('ignore')

## Retrieve and Inspect Capta

In [ ]:
os.getcwd()

In [ ]:
ts_df = pd.read_csv('../capta/processed/monthly_visitors_by_park.csv')
ts_df.head()

In [ ]:
ts_df.dt_pk = pd.to_datetime(ts_df.dt_pk)
ts_df = ts_df.set_index('dt_pk').asfreq('M', method='ffill')
ts_df.head()

## Simple Time Series Plots
- BIBE
- Faceted plots of all parks

In [ ]:
bibe_plot = (
    p9.ggplot(data=pd.DataFrame(ts_df['BIBE']), mapping=p9.aes(x='ts_df.index', y='BIBE'))
    + p9.geom_line()
    + p9.labs(
        x='Time',
        y='Monthly Visitors',
        title='Big Bend National Park'
    )
    + p9.theme_538()
    + p9.theme(axis_text_x=p9.element_text(rotation=90))
)
bibe_plot.draw();

In [ ]:
ts_df_r = ts_df.resample('Y').mean()
bibe_ann_plot = (
    p9.ggplot(data=pd.DataFrame(ts_df_r['BIBE']), mapping=p9.aes(x='ts_df_r.index', y='BIBE'))
    + p9.geom_line()
    + p9.labs(
        x='Time',
        y='Annual Avg. Monthly Visitors',
        title='Big Bend National Park'
    )
    + p9.theme_538()
    + p9.theme(axis_text_x=p9.element_text(rotation=90))
)
bibe_ann_plot.draw();

In [ ]:
bibe_plot = (
    p9.ggplot(data=pd.DataFrame(ts_df['BIBE']), mapping=p9.aes(x='ts_df.index', y='BIBE'))
    + p9.geom_line(color='#a6cee3')
    + p9.geom_line(data=pd.DataFrame(ts_df['BIBE']).resample('Y').mean(), mapping=p9.aes(x='ts_df.resample("Y").mean().index'), color='#1f78b4')
    + p9.labs(
        x='Time',
        y='Monthly Visitors',
        title='Big Bend National Park'
    )
    + p9.theme_538()
    + p9.theme(axis_text_x=p9.element_text(rotation=90))
)
bibe_plot.draw();

In [ ]:
melt_df = ts_df_r.reset_index().melt(id_vars='dt_pk', var_name='park', value_name='visitors').reset_index()
facet_plot = (
    p9.ggplot(data=melt_df, mapping=p9.aes(x='dt_pk', y='visitors'))
    + p9.geom_line()
    + p9.facet_wrap('park', scales='free_y')
    + p9.labs(
        x='Time',
        y='Annual Avg. Monthly Visitors',
        title='All National Parks Beginning with "B"'
    )
    + p9.theme_538()
    + p9.theme(figure_size=(8, 8), axis_text_x=p9.element_text(rotation=90), subplots_adjust={'wspace': 0.5})
)
facet_plot.draw();

## Modeling with AutoARIMA

In [ ]:
def ordinal(n):
    suffixes = {1: 'st', 2: 'nd', 3: 'rd'}
    return str(n) + suffixes.get(4 if 11 <= n % 100 < 14 else n % 10, 'th')

In [ ]:
def plot_ts(ts, lags=None, df_p_value=None, figsize=(12, 8)):
    """Produce a collection of artifacts for evaluating a time series.
    Adapted from https://towardsdatascience.com/multi-step-time-series-forecasting-with-arima-lightgbm-and-prophet-cc9e3f95dfb0#:~:text=ARIMA%20is%20one%20of%20the,p%2C%20d%2C%20and%20q.

    Args:
        ts (pd.Series): a time series
        lags (int): a number of lags to be passed to the ACF and PACF plots, instead
            of having this inferred automatically
        diffs (int): the number of times that differencing was applied before the DF test was significant
        df_p_value (float): the pre-computed result of a Dickey-Fuller test, if None then this will be calculated
        figsize (tuple): the figure size

    Returns:
        None
    """

    # Coerce input to series if not already
    if not isinstance(ts, pd.Series):
        ts = pd.Series(ts)

    # Create subpanels for subplots by creating a gridspec on top of an empty plotnine
    # figure (junk data is needed for backend "copy" reasons)
    fig = (p9.ggplot() + p9.geom_blank(data=pd.DataFrame(ts)) + p9.theme_void() + p9.theme(figure_size=figsize)).draw()
    gs = GridSpec(2, 2)
    ts_ax = fig.add_subplot(gs[0, 0:2])
    acf_ax = fig.add_subplot(gs[1, 0])
    pacf_ax = fig.add_subplot(gs[1, 1])

    # Plot of the time series itself
    if df_p_value is None:
        df_p_value = sm.tsa.stattools.adfuller(ts)[1]
    ts_plot = (
        p9.ggplot(data=pd.DataFrame(ts), mapping=p9.aes(x='ts.index', y=f'{ts.name}'))
        + p9.geom_line()
        + p9.labs(
            x='Time',
            y='Monthly Visitors'
        )
        + p9.theme_538()
        + p9.theme(axis_text_x=p9.element_text(rotation=90))
    )
    # A hack using a protected method is necessary to make plotnine work with matplotlib's subplot functionality
    _ = ts_plot._draw_using_figure(figure=fig, axs=[ts_ax])
    ts_ax.set_title(f'Time Series Analysis Plots for {ts.name}\nDickey-Fuller $p$-value: {df_p_value:.5}')

    # The other plots are created by statsmodels and fully compatible with matplotlib
    sm.tsa.graphics.plot_acf(ts, ax=acf_ax, lags=lags)
    sm.tsa.graphics.plot_pacf(ts, ax=pacf_ax, lags=lags)

    plt.tight_layout()

    fig.show()

In [ ]:
plot_ts(ts_df['BIBE'])

In [ ]:
# Differencing would appear to be in order
plot_ts((ts_df['BIBE'] - ts_df['BIBE'].shift(1)).dropna())

In [ ]:
# As would annual deseasoning
plot_ts((ts_df['BIBE'] - ts_df['BIBE'].shift(12)).dropna())

In [ ]:
deseasoned_ts = (ts_df['BIBE'] - ts_df['BIBE'].shift(12)).dropna()
plot_ts((deseasoned_ts - deseasoned_ts.shift(1)).dropna())

In [ ]:
ts = ts_df['BIBE'].dropna()
test_cutoff = int(0.8 * len(ts))
train_ts = ts[:test_cutoff]
test_ts = ts[test_cutoff:]

arima_model = AutoARIMA(m=12, max_order=8, trace=True)
arima_model.fit(train_ts)

In [ ]:
arima_model.summary().as_text()

In [ ]:
def plot_forecast(train_ts, test_ts, forecast, forecast_ci, train_limit=None, figure_size=(12, 4)):
    """Plots a forecast against the training capta and a holdout test set.

    Args:
        train_ts (pd.Series): the time series used to train the model
        test_ts (pd.Series): the holdout test portion of the time series
        forecast (array-like): the forecasts produced by an ARIMA model
        forecast_ci (array-like): the confidence interval for the forecasts
            produced by an ARIMA model
        train_limit (numeric): if present, the product of this number and the
            length of the test set will determine the maximum number of points
            in the training set to plot
        figure_size (tuple): the figure size

    Returns:
        None
    """

    # All series and forecasts should refer to the same phenomenon, and should
    # thus have the same name
    ts_name = train_ts.name
    mae = mean_absolute_error(test_ts, forecast)
    mape = mean_absolute_percentage_error(test_ts, forecast)

    # Subsample the training set for plotting if so indicated
    if train_limit is not None:
        train_limit = int(np.ceil(train_limit * len(test_ts)))
        # Ensure that there are sufficient capta to be plotted
        if not 0 < train_limit <= len(train_ts):
            raise ValueError('train_limit outside of acceptable bounds')
        train_ts = train_ts[-train_limit:]

    # Arrange all the capta into tidy objects for plotting - resetting all of
    # the indexes is necessary since the test and forecast time ranges coincide
    train_ts_df = pd.DataFrame(
        {ts_name: train_ts, 'forecast': False, 'split': 'train'}
    ).reset_index()
    test_ts_df = pd.DataFrame(
        {ts_name: test_ts, 'forecast': False, 'split': 'test'}
    ).reset_index()
    forecast_df = pd.DataFrame(
        {
            ts_name: forecast,
            'forecast': True,
            'split': 'forecast',
            'lower_bound': forecast_ci[:, 0],
            'upper_bound': forecast_ci[:, 1]
        },
        index=test_ts.index
    ).reset_index()
    plot_df = pd.concat(
        [train_ts_df, test_ts_df, forecast_df],
        ignore_index=True
    )

    # Filling NaNs is necessary so that plotnine doesn't drop the entire geom
    plot_df['lower_bound'] = plot_df['lower_bound'].fillna(plot_df[ts_name])
    plot_df['upper_bound'] = plot_df['upper_bound'].fillna(plot_df[ts_name])

    # Solid plot of the true values, dashed line plot of the forecast, and
    # ribbon plot of the forecase confidence interval
    forecast_plot = (
        p9.ggplot(
            data=plot_df,
            mapping=p9.aes(
                x='dt_pk',
                y=ts_name,
                color='split',
                linetype='forecast'
            )
        )
        + p9.geom_line()
        + p9.geom_ribbon(
            mapping=p9.aes(x='dt_pk', ymin='lower_bound', ymax='upper_bound'),
            fill='grey',
            alpha=0.5,
            inherit_aes=False
        )
        + p9.scale_color_brewer(type='qual', palette='Paired')
        + p9.labs(
            x='Time',
            y='Monthly Visitors',
            title=f'Forecast for {ts_name}\nMAE = {mae:.5}, MAPE = {mape:.5}'
        )
        + p9.theme_538()
        + p9.theme(
            figure_size=figure_size,
            axis_text_x=p9.element_text(rotation=90)
        )
    )
    forecast_plot.draw();

In [ ]:
forecast, forecast_ci = arima_model.predict(n_periods=len(test_ts), return_conf_int=True, alpha=0.05)
plot_forecast(train_ts, test_ts, forecast, forecast_ci)

## Extend to Additional Parks
- BLRI
- BOWA

In [ ]:
def plot_differenced_ts(ts, lags=None, diffs=0, df_p_value=None, figsize=(12, 8)):
    """Produce a collection of artifacts for evaluating a time series.
    Adapted from https://towardsdatascience.com/multi-step-time-series-forecasting-with-arima-lightgbm-and-prophet-cc9e3f95dfb0#:~:text=ARIMA%20is%20one%20of%20the,p%2C%20d%2C%20and%20q.

    Args:
        ts (pd.Series): a time series
        lags (int): a number of lags to be passed to the ACF and PACF plots, instead
            of having this inferred automatically
        diffs (int): the number of times that differencing was applied before the DF test was significant
        df_p_value (float): the pre-computed result of a Dickey-Fuller test, if None then this will be calculated
        figsize (tuple): the figure size

    Returns:
        None
    """

    # Coerce input to series if not already
    if not isinstance(ts, pd.Series):
        ts = pd.Series(ts)

    # Create subpanels for subplots by creating a gridspec on top of an empty plotnine
    # figure (junk data is needed for backend "copy" reasons)
    fig = (p9.ggplot() + p9.geom_blank(data=pd.DataFrame(ts)) + p9.theme_void() + p9.theme(figure_size=figsize)).draw()
    gs = GridSpec(2, 2)
    ts_ax = fig.add_subplot(gs[0, 0:2])
    acf_ax = fig.add_subplot(gs[1, 0])
    pacf_ax = fig.add_subplot(gs[1, 1])

    # Plot of the time series itself
    if df_p_value is None:
        df_p_value = sm.tsa.stattools.adfuller(ts)[1]
    ts_plot = (
        p9.ggplot(data=pd.DataFrame(ts), mapping=p9.aes(x='ts.index', y=f'{ts.name}'))
        + p9.geom_line()
        + p9.labs(
            x='Time',
            y='Monthly Visitors'
        )
        + p9.theme_538()
        + p9.theme(axis_text_x=p9.element_text(rotation=90))
    )
    # A hack using a protected method is necessary to make plotnine work with matplotlib's subplot functionality
    _ = ts_plot._draw_using_figure(figure=fig, axs=[ts_ax])
    ts_ax.set_title(f'Time Series Analysis Plots for Deseasoned {ordinal(diffs)}-Order Differenced {ts.name}\nDickey-Fuller $p$-value: {df_p_value:.5}')

    # The other plots are created by statsmodels and fully compatible with matplotlib
    sm.tsa.graphics.plot_acf(ts, ax=acf_ax, lags=lags)
    sm.tsa.graphics.plot_pacf(ts, ax=pacf_ax, lags=lags)

    plt.tight_layout()

    fig.show()

In [ ]:
parks = ['BLRI', 'BOWA']

for park in parks:
    ts = ts_df[park]
    test_cutoff = int(0.8 * len(ts))
    train_ts = ts[:test_cutoff]
    test_ts = ts[test_cutoff:]
    # NB: offload these to configs
    alpha = 0.05
    max_diffs = 12
    diffs = 0
    # Get rid of annual seasonality
    diff_ts = (train_ts - train_ts.shift(12)).dropna()
    df_p_value = sm.tsa.stattools.adfuller(diff_ts)[1]
    while alpha < df_p_value and diffs < max_diffs:
        diff_ts = (diff_ts - diff_ts.shift(1)).dropna()
        df_p_value = sm.tsa.stattools.adfuller(diff_ts)[1]
        diffs += 1
    plot_differenced_ts(diff_ts, diffs=diffs, df_p_value=df_p_value)

    arima_model = AutoARIMA(m=12, n_jobs=-1, max_order=8)
    arima_model.fit(train_ts)
    print(arima_model.summary())

    forecast, forecast_ci = arima_model.predict(n_periods=len(test_ts), return_conf_int=True, alpha=0.05)
    plot_forecast(train_ts, test_ts, forecast, forecast_ci)